In [32]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from PIL import Image
import cv2
from face_config import *

In [33]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [18]:
datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.15)

train_gen = datagen.flow_from_directory(
    directory = GENDER_DATA,
    subset = 'training',
    target_size= IMG_SIZE,
    color_mode=COLOR_MODE,
    batch_size= BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
)

val_gen = datagen.flow_from_directory(
    directory = GENDER_DATA,
    subset = 'validation',
    target_size= IMG_SIZE,
    color_mode=COLOR_MODE,
    batch_size= BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)


Found 20153 images belonging to 2 classes.
Found 3555 images belonging to 2 classes.


In [19]:
# transfer learning (needs rbg channels)
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False, weights='imagenet', input_shape=IMG_SHAPE, pooling="avg")                                          

# base_model.trainable = False

In [20]:
# model graph
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = base_model(inputs, training=False)
outputs = layers.Dense(units=2, activation="softmax", dtype='float32')(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 1280)              4049571   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2562      
Total params: 4,052,133
Trainable params: 4,010,110
Non-trainable params: 42,023
_________________________________________________________________


In [34]:
# Compile the model

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=LEARNING_RATE,
    decay_steps=10000,
    decay_rate=DECAY)

model.compile(loss=['binary_crossentropy'], 
                optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
                metrics=['accuracy'])

In [35]:
callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=LOG_PATH, histogram_freq=1),
        tf.keras.callbacks.ModelCheckpoint(MODEL_PATH,
                            monitor='accuracy',
                            verbose=1,
                            save_best_only=True,
                            mode='max')]

In [36]:
step_train = train_gen.n//train_gen.batch_size
step_val = val_gen.n//val_gen.batch_size

model.fit(train_gen, epochs=20, verbose = 1, callbacks = callbacks,
         validation_data = val_gen, steps_per_epoch = step_train, validation_steps = step_val)

Epoch 1/20
 53/314 [====>.........................] - ETA: 6:33 - loss: nan - accuracy: 0.5218

In [ ]:
model.save('gender_model.hdf5')